Set cuda

In [1]:
import torch
import os
import sys
device=0 # No other value works. known bug in Jupyter.

if torch.cuda.is_available():
    torch.cuda.set_device(device)

imports

In [2]:
%env WANDB_NOTEBOOK_NAME=JUPYTER

import wandb
import math

import os
from myhelpers import TrialStatistics
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from tqdm import tqdm
from tqdm.auto import trange
wandb.login()

env: WANDB_NOTEBOOK_NAME=JUPYTER


wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)


True

sweep params

In [3]:
# sweep_config = {
#     'method': 'random'
# }

sweep_config = {
    'method': 'bayes',
    'early_terminate': {
       'type': 'hyperband',
       'min_iter': 8   
    }
}
metric = {
    'name': 'best_val_f1',
    'goal': 'maximize'   
    }
sweep_config['metric'] = metric

In [4]:
project="CIFAR_KLDiv_Hyperp"
params = {
    "image_path":  {
        'value':"cifar-100-python"#"Curated4/Easy_50"
    },
    "suffix":{
        'value': None
    },
    
    # dataset
    "img_res": {
        'value': 32 #448
    }, 
    "augmented": {
        'values': [True, False]
    }, 

    # training
    "batchSize": {
          'values': [8, 16, 23, 64, 128, 256] # 8, 16, 64, 128
        }, 
    "learning_rate":{
        'distribution': 'log_uniform',
        'min': math.log(0.00005),
        'max': math.log(0.5),
      },
#     "learning_rate": { 'value': 0.00002},
    "numOfTrials":{
        'value': 1
    },
    "fc_layers": {
          'values': [1, 2, 3]
        },
#     "fc_layers": {'value': 2},
    "modelType":{
        'value':"BB"
    }, #BB DISCO DSN HGNN HGNN_add HGNN_cat
#     "lambda": {
#         'distribution': 'log_uniform',
#         'min': math.log(0.005),
#         'max': math.log(0.5),
#       },
    "lambda":{
        'value': 0
    },
    "unsupervisedOnTest": {
        'value': False
    },
    "tl_model": {
        'value': 'CIFAR'#'ResNet18'
    },
    "link_layer": {
        'value': 'avgpool'
    },
#     "link_layer":  {
#         'values': ["avgpool","layer3"]
#     }, 
    
#     "adaptive_smoothing": {
#         'values': [True,False]
#     },
    "adaptive_smoothing": {
        'value': False
    },
    "adaptive_lambda": {
        'value': 0.001
    },
#     "adaptive_lambda": {
#         'distribution': 'log_uniform',
#         'min': math.log(0.001),
#         'max': math.log(0.1),
#       }, 
#     "adaptive_alpha": {
#         'min': 0.5,
#         'max': 0.99,
#       }, 
    "adaptive_alpha": {
        'value': 0.5
    },
    
#     "noSpeciesBackprop":{
#         'values': [True,False]
#     },
    "noSpeciesBackprop": {
        'value': False
    },
    "phylogeny_loss":{
        'values': ["KLDiv"]#[False, "MSE", "KLDiv"] # False, "MSE", "KLDiv"
    },
#     "phylogeny_loss_epsilon": {
#         'distribution': 'log_uniform',
#         'min': math.log(0.001),
#         'max': math.log(1),
#       }, 
    "phylogeny_loss_epsilon": {
        'value': 0.03
    },
}

sweep_config['parameters'] = params

train function

In [5]:

# python3 HGNN/code/HGNN/HGNN/train/train.py --cuda=6 --name="biology_paper_curated4_Medium_backprop_experiment" -- -- --detailed

experimentsPath = "/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = project

def train(config=None):
    run = wandb.init()

#     try:
    experiment_params=wandb.config if config is None else config
    experiment_params = dict(experiment_params)
    config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
    print(experiment_params)
    experimentHash =TrialStatistics.getTrialName(experiment_params)

    # load images 
    experimentPathAndName = os.path.join(experimentsPath, experimentName)
    if experiment_params['image_path'] == 'cifar-100-python':
        datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
    else:
        datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
    datasetManager.updateParams(config_parser.fixPaths(experiment_params))
    train_loader, validation_loader, test_loader = datasetManager.getLoaders()
    architecture = {
        "fine": len(train_loader.dataset.csv_processor.getFineList()),
        "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
    }

    # Loop through n trials
    for i in trange(experiment_params["numOfTrials"], desc="trial"):
        modelName = getModelName(experiment_params, i)
        trialName = os.path.join(experimentPathAndName, modelName)
        trialHash = TrialStatistics.getTrialName(experiment_params, i)

        row_information = {
            'experimentName': experimentName,
            'modelName': modelName,
            'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
            'experimentHash': experimentHash,
            'trialHash': trialHash
        }
        row_information = {**row_information, **experiment_params} 
        print(row_information)


        # Train/Load model
        model = CNN.create_model(architecture, experiment_params, device=device)

        if os.path.exists(CNN.getModelFile(trialName)):
            print("Model {0} found!".format(trialName))
        else:
            CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=False)

    run.finish()
#     except:
#         print('terminated! Error:', sys.exc_info()[0])

Call sweeper!

In [6]:
# You can call with your own configs if you want:
# train(config={...})

In [7]:
# Or call the sweeper and let it do the magic!
sweep_id = wandb.sweep(sweep_config, project=project)

Create sweep with ID: tquv2ll1
Sweep URL: https://wandb.ai/mndhamod/CIFAR_KLDiv_Hyperp/sweeps/tquv2ll1


In [ ]:
wandb.agent(sweep_id, function=train, count=100)
# !CUDA_VISIBLE_DEVICES=1 wandb agent k7cnzayi --count 100 #train.py from program?

wandb: Agent Starting Run: c6pxep5o with config:
wandb: 	adaptive_alpha: 0.5
wandb: 	adaptive_lambda: 0.001
wandb: 	adaptive_smoothing: False
wandb: 	augmented: False
wandb: 	batchSize: 8
wandb: 	fc_layers: 1
wandb: 	image_path: cifar-100-python
wandb: 	img_res: 32
wandb: 	lambda: 0
wandb: 	learning_rate: 0.08200490621168555
wandb: 	link_layer: avgpool
wandb: 	modelType: BB
wandb: 	noSpeciesBackprop: False
wandb: 	numOfTrials: 1
wandb: 	phylogeny_loss: KLDiv
wandb: 	phylogeny_loss_epsilon: 0.03
wandb: 	suffix: None
wandb: 	tl_model: CIFAR
wandb: 	unsupervisedOnTest: False
wandb: wandb version 0.10.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'adaptive_alpha': 0.5, 'adaptive_lambda': 0.001, 'adaptive_smoothing': False, 'augmented': False, 'batchSize': 8, 'fc_layers': 1, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.08200490621168555, 'link_layer': 'avgpool', 'modelType': 'BB', 'noSpeciesBackprop': False, 'numOfTrials': 1, 'phylogeny_loss': 'KLDiv', 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'unsupervisedOnTest': False}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Creating dataset... Done.
Loading saved indices...
Creating loaders...
Creating loaders... Done.


{'experimentName': 'CIFAR_KLDiv_Hyperp', 'modelName': 'models/3e2d532c9da7f6c513b8d4d0225243afe57dcfde93f0183a6ea84fbc', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': 'f18bc1e0246d0e6260f3542aff2cee9ef8d94f1f621771fc1739fbb7', 'trialHash': '3e2d532c9da7f6c513b8d4d0225243afe57dcfde93f0183a6ea84fbc', 'adaptive_alpha': 0.5, 'adaptive_lambda': 0.001, 'adaptive_smoothing': False, 'augmented': False, 'batchSize': 8, 'fc_layers': 1, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.08200490621168555, 'link_layer': 'avgpool', 'modelType': 'BB', 'noSpeciesBackprop': False, 'numOfTrials': 1, 'phylogeny_loss': 'KLDiv', 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'unsupervisedOnTest': False}


iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Training started...


...

{'adaptive_alpha': 0.5, 'adaptive_lambda': 0.001, 'adaptive_smoothing': False, 'augmented': True, 'batchSize': 8, 'fc_layers': 2, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.00017646943262504745, 'link_layer': 'avgpool', 'modelType': 'BB', 'noSpeciesBackprop': False, 'numOfTrials': 1, 'phylogeny_loss': 'KLDiv', 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'unsupervisedOnTest': False}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Creating dataset... Done.
Loading saved indices...
Creating loaders...
Creating loaders... Done.


{'experimentName': 'CIFAR_KLDiv_Hyperp', 'modelName': 'models/1d4400e9769755e40ae25bf0022c9b4a7006cf0a944defc32e9fbc2f', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '66f3539a6b90747edf7cdee48877e870880c1e99e252d46d94a93db0', 'trialHash': '1d4400e9769755e40ae25bf0022c9b4a7006cf0a944defc32e9fbc2f', 'adaptive_alpha': 0.5, 'adaptive_lambda': 0.001, 'adaptive_smoothing': False, 'augmented': True, 'batchSize': 8, 'fc_layers': 2, 'image_path': 'cifar-100-python', 'img_res': 32, 'lambda': 0, 'learning_rate': 0.00017646943262504745, 'link_layer': 'avgpool', 'modelType': 'BB', 'noSpeciesBackprop': False, 'numOfTrials': 1, 'phylogeny_loss': 'KLDiv', 'phylogeny_loss_epsilon': 0.03, 'suffix': None, 'tl_model': 'CIFAR', 'unsupervisedOnTest': False}


iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Training started...


iteration:   2%|▏         | 8/500 [1:15:31<76:18:13, 558.32s/it, min_val_loss=5.98, train=0.174, val=0.163, val_loss=4.58]

In [ ]:
# sum([1.5041e-03, 1.2033e-03, 1.1173e-03, 1.5041e-03, 2.8363e-03, 1.3752e-03,
#          1.3322e-03, 2.3636e-03, 2.2776e-03, 2.3636e-03, 2.2346e-03, 2.3206e-03,
#          2.3206e-03, 2.4065e-03, 2.4065e-03, 1.9338e-03, 1.9768e-03, 3.8676e-04,
#          5.1569e-04, 4.2974e-04, 3.4379e-04, 4.7271e-04, 0.0000e+00, 8.5948e-05,
#          2.5784e-04, 4.7271e-04, 4.2974e-04, 4.2974e-04, 2.5784e-04, 4.7271e-04,
#          3.4379e-04, 4.7271e-04, 1.4181e-03, 1.6330e-03, 1.5471e-03, 1.6760e-03,
#          1.5041e-03, 3.0082e-04])

In [ ]:


[x / 1362 for x in [30., 31., 29., 30., 61., 27., 26., 50., 48., 50., 47., 49., 49., 51.,
         51., 40., 41., 40., 43., 41., 39., 42., 33., 33., 37., 42., 41., 41.,
         37., 42., 39., 42.,  0.,  7.,  5.,  8.,  4., 36.]]